# How similar is the text?

In [1]:
import os, sys, spacy

import pandas as pd
# import gensim.downloader as api

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from feature_extraction import SpacyFeatureExtraction
from data_processing import DataProcessing
from clean_predictions import PredictionDataCleaner

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/detraviousjamaribrinkley/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load_nlp_model = spacy.load("en_core_web_md")
# pretrained_word_two_vec_model = api.load('word2vec-google-news-300')

In [3]:
mappings = {
    "I predict it will be sunny on 4 April 2025." : ["It was sunny on 4 April 2025.", "It was not sunny on 4 April 2025."],
    "Apple's stock will rise by 10%." : ["Apple's stock rose by 11%.", "Apple's stock rose by 900%.", "Apple's stock rose by 10%."]
}

mappings = {
    "Tesla's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 10%.", "Apple's stock rose by 900%."]
}

mappings = {
    "Tesla's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 10%.", "Apple's stock rose by 900%."],
    "Apple's stock will rise by 10%." : ["Apple's stock will rise by 10%.", "Apple's stock rose by 11%.", "Apple's stock rose by 900%.", "Apple's stock rose by 10%."]
}

In [4]:
load_nlp_model = spacy.load("en_core_web_sm")

all_mappings = {} # Mapping of predictions to observations
pom_mappings = {} # Mapping of predictions to observations to metrics/scoresl pos can be misleading

oms = []
for prediction in mappings.keys():
    print(f"Prediction: {prediction}")

    om_mappings = {} # Mapping of observations to metrics/scores

    for observation in mappings[prediction]:
        print(f"    Observation: {observation}")

        # Calculate the similarity score
        prediction_doc = load_nlp_model(prediction)
        observation_doc = load_nlp_model(observation)
        similarity = prediction_doc.similarity(observation_doc)
        print(f"    Similarity: {similarity}")
        om_mappings[observation] = [similarity]
        print()   
        # print(f"    OM Mapping: {om_mappings}")
    pom_mappings[prediction] = om_mappings


/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_602/312081125.py:18: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc)


Prediction: Tesla's stock will rise by 10%.
    Observation: Apple's stock will rise by 10%.
    Similarity: 0.9796735644340515

    Observation: Apple's stock rose by 11%.
    Similarity: 0.8603723049163818

    Observation: Apple's stock rose by 10%.
    Similarity: 0.8691261410713196

    Observation: Apple's stock rose by 900%.
    Similarity: 0.8487552404403687

Prediction: Apple's stock will rise by 10%.
    Observation: Apple's stock will rise by 10%.
    Similarity: 1.0

    Observation: Apple's stock rose by 11%.
    Similarity: 0.8647665977478027

    Observation: Apple's stock rose by 900%.
    Similarity: 0.8541026711463928

    Observation: Apple's stock rose by 10%.
    Similarity: 0.8756679892539978



In [5]:
# pom_mappings

In [6]:
# Flatten the dictionary into a list of rows
data = []
for prediction, observations in pom_mappings.items():
    for observation, scores in observations.items():
        data.append([prediction, observation, scores[0]])

# Create the DataFrame
df = pd.DataFrame(data, columns=['Prediction', 'Observation', 'Spacy Similarity'])

# Display the DataFrame
df

,Prediction,Observation,Spacy Similarity
0,Tesla's stock will rise by 10%.,Apple's stock will rise by 10%.,0.979674
1,Tesla's stock will rise by 10%.,Apple's stock rose by 11%.,0.860372
2,Tesla's stock will rise by 10%.,Apple's stock rose by 10%.,0.869126
3,Tesla's stock will rise by 10%.,Apple's stock rose by 900%.,0.848755
4,Apple's stock will rise by 10%.,Apple's stock will rise by 10%.,1.000000
5,Apple's stock will rise by 10%.,Apple's stock rose by 11%.,0.864767
6,Apple's stock will rise by 10%.,Apple's stock rose by 900%.,0.854103
7,Apple's stock will rise by 10%.,Apple's stock rose by 10%.,0.875668


Comparison
1. No focus on #s, more on words

In [16]:
prediction_doc = load_nlp_model("10%")
observation_doc = load_nlp_model("10.5%")
observation_doc_2 = load_nlp_model("11%")
similarity = prediction_doc.similarity(observation_doc)
similarity

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_602/1801851099.py:4: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc)


0.925104558467865

In [17]:
similarity = prediction_doc.similarity(observation_doc_2)
similarity

/var/folders/78/9z0b45fx1xqbwxh8vk97lcfh0000gn/T/ipykernel_602/4256136358.py:1: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity = prediction_doc.similarity(observation_doc_2)


0.9595414400100708

In [18]:
# prediction_doc = load_nlp_model("Apple's stock rose by 10%.")
# observation_doc = load_nlp_model("Apple's stock rose by 11%.")
# similarity = prediction_doc.similarity(observation_doc)
# similarity

In [19]:
pdv = prediction_doc.vector # 10
pdv

array([ 0.30040288, -0.46711448,  0.509174  ,  0.1586005 ,  1.1491344 ,
       -0.8531876 ,  0.50995904,  0.27860662, -0.3254361 , -0.1560917 ,
       -0.26071814,  0.29767483, -0.39653268,  1.1135507 , -0.2526727 ,
        0.36130965, -0.81857663, -0.40923283, -1.0234685 ,  0.37592494,
       -0.3877185 ,  0.78387195,  0.87653255,  0.03448987,  0.56956613,
        0.1211684 ,  0.10060082, -0.39651704,  0.03264901,  0.2642506 ,
        0.9213592 ,  0.03562765,  0.290687  , -0.16653627, -0.8325365 ,
       -0.052293  , -0.7682663 , -0.4528045 ,  0.31796303,  0.5184007 ,
       -0.36622745, -0.08055362, -0.50146323,  1.4582404 , -0.08100343,
       -0.17221056, -0.84047353,  0.74705327, -0.02882719, -0.24370924,
        0.57494605, -0.44427025,  0.3779632 , -0.6967993 ,  1.6423988 ,
       -0.22711053,  0.22832489, -0.09112316,  0.3020852 ,  0.06474298,
       -0.32058564, -0.47859693, -0.8530745 , -1.0867817 , -0.07193637,
       -0.18350723, -0.02099572, -0.16591716, -0.6374334 ,  0.68

In [20]:
odv_1 = observation_doc.vector # 10.5
odv_1

array([ 0.04759576, -0.3010687 ,  0.13733122,  0.09303613,  1.5409473 ,
       -0.7031069 ,  0.42882633,  0.02008379,  0.05921438, -0.26620144,
       -0.1433432 ,  0.22777021, -0.44027084,  1.2943797 , -0.02111381,
        0.38246828, -0.65070444,  0.06801617, -0.87427586,  0.17526266,
       -0.38326454,  1.0280204 ,  0.57325053, -0.11215457,  0.64881575,
        0.2443167 , -0.21498092, -0.6615258 ,  0.28375006,  0.20735359,
        0.81387967, -0.24033983,  0.37064815, -0.39679313, -0.9169711 ,
        0.20031974, -0.4901891 , -0.23531848,  0.32470727,  0.6202681 ,
        0.08728933, -0.25813505, -0.7733021 ,  1.5522578 ,  0.16665655,
       -0.37733817, -0.8878894 ,  0.6666155 ,  0.16304165,  0.08384445,
        0.19909966, -0.05296937,  0.3485108 , -0.93762916,  1.8119321 ,
       -0.13450277,  0.20743313,  0.05026382,  0.3674773 ,  0.24333894,
       -0.31718045, -0.64950943, -0.65276927, -0.98642814,  0.01255167,
       -0.24843585, -0.31948543, -0.01021798, -0.77451   ,  0.41

[Five most popular similarity measures implementation in python](https://dataaspirant.com/five-most-popular-similarity-measures-implementation-in-python/)

In [21]:
odv_2 = observation_doc_2.vector # 11
odv_2

array([ 0.23161265, -0.43362677,  0.37865746,  0.12842351,  1.474714  ,
       -0.74072057,  0.60628223,  0.2636813 , -0.17834927, -0.18636319,
       -0.31557438,  0.19734126, -0.19305493,  1.1006242 , -0.0965907 ,
        0.298085  , -0.93780375, -0.06959879, -0.84290695,  0.3828358 ,
       -0.51774096,  1.064464  ,  0.7960936 ,  0.064152  ,  0.5760752 ,
       -0.06326589, -0.01081962, -0.44413358,  0.12883908,  0.17736298,
        0.7629027 , -0.13168155,  0.16614637, -0.35794282, -0.727747  ,
       -0.08037841, -0.6158122 , -0.37415862,  0.30544767,  0.42032158,
       -0.2694553 , -0.28181946, -0.7353734 ,  1.510866  ,  0.07865137,
       -0.27992103, -0.8563335 ,  0.8413692 ,  0.03110853, -0.24775779,
        0.36043683, -0.25667107,  0.33579728, -0.8342086 ,  1.9182353 ,
        0.05007973,  0.3324153 ,  0.14967984,  0.43167794,  0.09882894,
       -0.50839734, -0.63794184, -0.837104  , -0.95582104, -0.12988025,
       -0.0932615 , -0.08784196, -0.10974275, -0.82919765,  0.36

- Pearson and Spearman Correlations
    - 1 perfect positive relationship
    - 0 no relationship
    -  -1 perfect negative relationshp
- Reference: https://mccormickml.com/2019/11/05/GLUE/

In [22]:
from scipy import stats
p_result_1 = stats.pearsonr(pdv, odv_1)
p_result_2 = stats.pearsonr(pdv, odv_2)

s_result_1 = stats.spearmanr(pdv, odv_1)
s_result_2 = stats.spearmanr(pdv, odv_2)

p_result_1, p_result_2, s_result_1, s_result_2

(PearsonRResult(statistic=np.float32(0.9250324), pvalue=np.float64(2.722438055783015e-41)),
 PearsonRResult(statistic=np.float32(0.9595269), pvalue=np.float64(1.5838307052880837e-53)),
 SignificanceResult(statistic=np.float64(0.8928648941942485), pvalue=np.float64(2.4757606347787834e-34)),
 SignificanceResult(statistic=np.float64(0.9481551817688552), pvalue=np.float64(1.3813170610308815e-48)))

In [ ]:
from math import*

def square_rooted(x):

    return round(sqrt(sum([a*a for a in x])),3)

def cosine_similarity(x,y):

    numerator = sum(a*b for a,b in zip(x,y))
    denominator = square_rooted(x)*square_rooted(y)
    return round(numerator/float(denominator),3)

cosine_similarity(pdv, odv)

In [ ]:
prediction_doc = load_nlp_model("10")
observation_doc = load_nlp_model("10.5")
observation_doc_2 = load_nlp_model("11")

similarity = prediction_doc.similarity(observation_doc)
similarity

In [ ]:
similarity = prediction_doc.similarity(observation_doc_2)
similarity

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example sentences
sentence1 = "Apple's stock rose by 10%."
sentence2 = "Apple's stock rose by 10.5%."

# Tokenize the sentences
tokens1 = tokenizer.tokenize(sentence1)
tokens2 = tokenizer.tokenize(sentence2)

# Add [CLS] and [SEP] tokens
tokens = ['[CLS]'] + tokens1 + ['[SEP]'] + tokens2 + ['[SEP]']
print("Token:", tokens)

# Convert tokens to input IDs
input_ids = tokenizer.convert_tokens_to_ids(tokens)

# Display the tokens and input IDs
print("Input IDs:", input_ids)

from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Convert tokens to input IDs
input_ids1 = torch.tensor(tokenizer.convert_tokens_to_ids(tokens1)).unsqueeze(0)  # Batch size 1
input_ids2 = torch.tensor(tokenizer.convert_tokens_to_ids(tokens2)).unsqueeze(0)  # Batch size 1

# Obtain the BERT embeddings
with torch.no_grad():
    outputs1 = model(input_ids1)
    outputs2 = model(input_ids2)
    embeddings1 = outputs1.last_hidden_state[:, 0, :]  # [CLS] token
    embeddings2 = outputs2.last_hidden_state[:, 0, :]  # [CLS] token

# Calculate similarity
similarity_score = cosine_similarity(embeddings1, embeddings2)
print("Similarity Score:", similarity_score)

In [ ]:
mappings = {
    "Prediction": ["I am predicting it will be sunny on 4 April 2025.", "Apple's stock will be rising by 10%.", "Apple's stock will be rising by 10%."],
    "Observation": ["It was sunny on 4 April 2025", "Apple's stock is rising by 900%.",  "Apple's stock rose by 900%."]
}

df = pd.DataFrame(mappings)
df


In [ ]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

clean_data = PredictionDataCleaner(df)

In [ ]:
updated_predictions_df = clean_data.lemmatize(col_name='Prediction', lem_col_name='Predictions Lem')
updated_predictions_df

In [ ]:
updated_observations_df = clean_data.lemmatize(col_name='Observation', lem_col_name='Observations Lem')
updated_observations_df

In [ ]:
lem_df = updated_predictions_df.merge(updated_observations_df)
lem_df

- Issues above are: 

    1. "'s" isn't joined to Apple
    2. It doesn't consider "will be rising" as a single future tense phrase, thus breaks it up into words (with only one future tense word of "rising")

In [ ]:
import nltk
nltk.download('wordnet')
  
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

lem = WordNetLemmatizer()

In [ ]:
lem.lemmatize('predicting')

In [ ]:
lem.lemmatize('rising')

In [ ]:
lem.lemmatize('rocks')

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("predicting, predicts, rising, rises, rocks, rose, will be rising, is rising by.")
doc = nlp("I am predicting it will be sunny on 4 April 2025.")

words_lemmas_list = [token.lemma_ for token in doc]
print(words_lemmas_list)
